In [17]:
from jax import make_jaxpr
import jax.numpy as jnp
from jax import grad, jit, vmap
import jax.random as random
key = random.PRNGKey(0)

def k():
    global key
    key, key2 = random.split(key)
    return key2

In [18]:
def func1(first, second):
    temp = first + jnp.sin(second) * 3.
    return jnp.sum(temp)

make_jaxpr(func1)(jnp.zeros(8), jnp.ones(8))


{ lambda ; a:f32[8] b:f32[8]. let
    c:f32[8] = sin b
    d:f32[8] = mul c 3.0
    e:f32[8] = add a d
    f:f32[] = reduce_sum[axes=(0,)] e
  in (f,) }

In [26]:
x = random.normal(k(), (8,))
y = random.normal(k(), (8,))
make_jaxpr(grad(func1, 1))(x, y)

{ lambda ; a:f32[8] b:f32[8]. let
    c:f32[8] = sin b
    d:f32[8] = cos b
    e:f32[8] = mul c 3.0
    f:f32[8] = add a e
    _:f32[] = reduce_sum[axes=(0,)] f
    g:f32[8] = broadcast_in_dim[broadcast_dimensions=() shape=(8,)] 1.0
    h:f32[8] = mul g 3.0
    i:f32[8] = mul h d
  in (i,) }

In [27]:
print(func1(x, y))
print(grad(func1, 1)(x, y))

-0.5880289
[-1.0148692  2.9654224  1.3846326  1.0471934  2.8733923  1.1474879
  2.9817548  2.9977417]


In [16]:
k()

(DeviceArray([2384771982, 3928867769], dtype=uint32),
 DeviceArray([1278412471, 2182328957], dtype=uint32))